# Model Calibration

This notebook was made to help calibrate the model. It is being retained because its outputs may be important.

Run this cell before running any other parts:

In [15]:
from config import *
import pandas as pd
import matplotlib.pyplot as plt

## Risk component daymet point extractions

This section will extract the daymet-derived risk components for a subset of locations.

Define the locations and years we are interested in:

In [2]:
import numpy as np


years = list(range(2010, 2018))
locations_lu = {
    "Talkeetna": {
        "years": years,
        "latlon": (62.3209, -150.1066),
    },
    "Fairbanks": {
        "years": years,
        "latlon": (64.8401, -147.7200),
    },
    "King Salmon": {
        "years": years,
        "latlon": (58.6887, -156.6628),
    },
    "Delta Jct": {
        "years": years,
        "latlon": (64.0401, -145.7344),
    },
    
    "Wasilla": {
        "years": years,
        "latlon": (61.5809, -149.4411),
    },
    "Fort Yukon": {
        "years": years,
        "latlon": (66.5637, -145.2487),
    },
    "Tok": {
        "years": years,
        "latlon": (63.3359, -142.9877),
    },
    "Glenallen": {
        "years": years,
        "latlon": (62.1081, -145.5340),
    },
}

Define functions to extract the risk values from the yearly risk dataset:

In [3]:
def latlon_to_idx(da, lat, lon):
    if da["longitude"].values.max() > 0:
        # create new longitude array for point extraction that 
        #  is on the [-360, 0) scale in case it's not
        new_lon = da["longitude"].values.copy()
        new_lon[new_lon > 0] = new_lon[new_lon > 0] - 360
    else:
        new_lon = ds["longitude"].values
    
    dist_arr = np.sqrt(
        np.square(new_lon - lon) + np.square(da["latitude"] - lat)
    ).values

    yidx, xidx = np.where(dist_arr == dist_arr.min())
    
    return yidx, xidx


def extract_risk_df(da, lat, lon, years):
    """Extract a dataframe of risk values for a given location"""
    yidx, xidx = latlon_to_idx(da, lat, lon)
    risk_df = da.sel(y=yidx, x=xidx, year=years).drop(["latitude", "longitude"]).to_dataframe(
        "risk"
    ).reset_index().drop(columns=["x", "y"])
    
    return risk_df

Define a function to extract the risk components involvied in a year's risk computation for a given location:

In [10]:
def get_components(daymet_comp_fp, year, snow, lat, lon):
    components = {}
    with xr.open_dataset(daymet_comp_fp) as comp_ds:
        yidx, xidx = latlon_to_idx(comp_ds, lat, lon)
        const_args = {"y": yidx, "x": xidx}
        
        components["u_t2"] = comp_ds["summer_survival"].sel(year=(year - 2), **const_args).values[0][0]
        components["u_t1"] = comp_ds["summer_survival"].sel(year=(year - 1), **const_args).values[0][0]
        # "not univoltine"
        components["un_t2"] = np.round(1 - components["u_t2"], 2)
        components["x2_t2"] = comp_ds["fall_survival"].sel(year=(year - 2), **const_args).values[0][0]
        components["x2_t1"] = comp_ds["fall_survival"].sel(year=(year - 1), **const_args).values[0][0]
        components["x3_t2"] = comp_ds["winter_survival"].sel(year=(year - 2), snow=snow, **const_args).values[0][0]
        components["x3_t1"] = comp_ds["winter_survival"].sel(year=(year - 1), snow=snow, **const_args).values[0][0]

    return components

Apply the functions and create a complete dataframe of extracted values for all desired locations and years for the daymet risk components and yearly risk datasets:

In [11]:
yearly_risk_fp = yearly_risk_dir.joinpath("yearly_risk_daymet_1980-2017.nc")
daymet_comp_fp = risk_comp_dir.joinpath("risk_components_daymet_1980-2017.nc")
with xr.open_dataset(yearly_risk_fp) as risk_ds:
    risk_df_list = []
    # start of iteration over locations
    for location in locations_lu:
        years = locations_lu[location]["years"]
        lat, lon = locations_lu[location]["latlon"]
        # ensure years are limited to daymet availability (up to 2017)
        years = [year for year in years if year <= 2017]
        temp_df = extract_risk_df(risk_ds["risk"], lat, lon, years)
        temp_df["location"] = location
        
        temp_components = []
        for idx, row in temp_df.iterrows():
            snow = row["snow"]
            year = row["year"]
            temp_components.append(get_components(daymet_comp_fp, year, snow, lat, lon))
        temp_df["components"] = temp_components
        
        risk_df_list.append(temp_df)

risk_df = pd.concat(risk_df_list)
risk_df = risk_df[["location"] + list(risk_df.columns[:-2]) + ["components"]]

comp_df = risk_df["components"].apply(pd.Series)
risk_df = risk_df.drop(columns="components")
risk_df[comp_df.columns] = comp_df

Write the CSV

In [13]:
for comp in ["u_t2", "u_t1", "un_t2", "x2_t2", "x2_t1", "x3_t2", "x3_t1"]:
    risk_df[comp] = risk_df[comp].map(lambda x: '%.2f' % x)
risk_table_fp = scratch_dir.joinpath("daymet_risk_extraction.csv")
risk_df.to_csv(risk_table_fp, index=False, float_format='%.6f')

## Daymet yearly risk maps

Create risk maps for all years of Daymet data using the yearly risk dataset.

Load the Daymet yearly risk data:

In [16]:
daymet_risk_fp = yearly_risk_dir.joinpath("yearly_risk_daymet_1980-2017.nc")
ds = xr.load_dataset(daymet_risk_fp)

Create output directory:

In [18]:
out_dir = scratch_dir.joinpath("daymet_risk_maps")
out_dir.mkdir(exist_ok=True)

Iterate, create the maps, and save them:

In [19]:
for year in range(1982, 2018):
    fig, ax = plt.subplots(1, 1, figsize=(18, 8))
    arr = ds["risk"].sel(snow="med", year=year).values
    im = ax.imshow(arr, interpolation="none")
    plt.axis("off")
    cbar_ax = fig.add_axes([0.7, 0.15, 0.02, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.ax.get_yaxis().labelpad = 30
    cbar.set_label("Beetle\nrisk", size=14, rotation=0)
    ax.set_title(f"Beetle risk (Daymet), {year}", size=14, pad=-5)
    out_fp = out_dir.joinpath(f"daymet_risk_{year}.png")
    plt.savefig(out_fp, bbox_inches='tight', facecolor="white")
    plt.close()